In [ ]:
import pandas as pd
import psycopg2
import numpy as np
import re
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO)

# Load the JSON data into a pandas DataFrame
df = pd.read_json('amazon_data_ext.json')

# Replace NaN values in specified string columns with "None"
string_columns = [
    'Customer_1_ID',
    'Customer_1_Comment',
    'Customer_id_Critical_Review',
    'Customer_Name',
    'Customer_Review_Comment',
    'Customer_Review_Title',
    'Customer_2_ID'
]
df[string_columns] = df[string_columns].fillna("None")

# Replace NaN values in specified integer columns with 0
integer_columns = [
    'Customer_1_Star_Rating',
    'Customer_1_buying_influence',
    'Customers_Influenced'
]
df[integer_columns] = df[integer_columns].fillna(0)

# Check if the 'Customers_Influenced' column is in the DataFrame
if 'Customers_Influenced' not in df.columns:
    print("Column 'Customers_Influenced' not found in the DataFrame. Please check the column name in the JSON file.")

# Convert 'Post_Date' to datetime object and handle any conversion errors by setting them to NaT
df['Post_Date'] = pd.to_datetime(df['Post_Date'], errors='coerce', format='%Y-%m-%dT%H:%M:%S')

# Replace NaT values with a default date string '0001-01-01'
df['Post_Date'].fillna(pd.Timestamp('0001-01-01'), inplace=True)

# Convert 'Post_Date' to 'yyyy-mm-dd' string format for PostgreSQL
df['Post_Date'] = df['Post_Date'].dt.strftime('%Y-%m-%d')


# Replace NaN values with 'None' in the specified columns
df['Customer_id_Critical_Review'] = df['Customer_id_Critical_Review'].replace({'"NaN"': 'None', 'NaN': 'None'})
df['Customer_Name'] = df['Customer_Name'].replace({'"NaN"': 'None', 'NaN': 'None'})
df['Customer_Review_Comment'] = df['Customer_Review_Comment'].replace({'"NaN"': 'None', 'NaN': 'None'})
df['Customer_Review_Title'] = df['Customer_Review_Title'].replace({'"NaN"': 'None', 'NaN': 'None'})
df['Customers_Influenced'] = df['Customers_Influenced'].replace({'"NaN"': 'None', 'NaN': 'None'})

# Define the columns to be updated
columns_to_update = [f'Customer_{i}_Comment' for i in range(1, 6)]
for column in columns_to_update:
    df[column] = df[column].replace({'"NaN"': 'None', 'NaN': 'None'})

columns_to_update = [f'Customer_{i}_ID' for i in range(1, 6)]
for column in columns_to_update:
    df[column] = df[column].replace({'"NaN"': 'None', 'NaN': 'None'})

# Drop all items with review having zero values
df.drop(df.index[df['Customer_1_ID'] == 'None'], inplace=True)

# Define the columns to be updated
new_columns = ['Customer_id_Critical_Review', 'Customer_Name', 'Customer_Review_Comment', 'Customer_Review_Title', 'Customers_Influenced']
for column in new_columns:
    if column == 'Customers_Influenced': df[column] = df[column].replace({'"NaN"': 'None', 'NaN': 'None'})

# Drop the 'review_responders' column if it exists
if 'review_responders' in df.columns:
    df.drop(columns=['review_responders'], inplace=True)

# Handle other columns similarly
df['price'] = df['price'].apply(lambda x: float(x) if pd.notnull(x) else None)
df['ratings'] = df['ratings'].apply(lambda x: float(x) if pd.notnull(x) else None)
df['reviews'] = df['reviews'].fillna(0).astype(int)

# Drop rows where the 'price' column is NaN
df.dropna(subset=['price'], inplace=True)

# Drop all items with review having zero values or Customer_1_ID as 'None'
df.drop(df.index[(df['reviews'] == 0) | (df['Customer_1_ID'] == 'None')], inplace=True)

################################
# df.info()
# print(df.tail())
################################
# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="demopass",
    client_encoding='utf8'
)

cur = conn.cursor()

# Modify the CREATE TABLE query to include additional columns
create_table_query = """
DROP TABLE IF EXISTS amazon_data_ext;
CREATE TABLE IF NOT EXISTS amazon_data_ext (
    product_id TEXT NOT NULL,
    product TEXT NOT NULL,
    star_ratings NUMERIC NULL,
    price_dollars NUMERIC NULL,
    total_ratings INTEGER NOT NULL,
    category TEXT NOT NULL,
    url TEXT NOT NULL,
    Customer_id_Critical_Review TEXT,
    Customer_Name TEXT,
    Post_Date DATE,
    Customer_Review_Comment TEXT,
    Customer_Review_Title TEXT,
    Customers_Influenced INTEGER,
    """ + ",\n    ".join([f"Customer_{i}_ID TEXT, Customer_{i}_Star_Rating NUMERIC, Customer_{i}_Comment TEXT, Customer_{i}_buying_influence INTEGER" for i in range(1, 6)]) + """
)
"""
cur.execute(create_table_query)
conn.commit()

def clean_format_data(row):
    # Extract the values directly, as they are already cleaned
    ratings = row['ratings']
    price = row['price']
    reviews = row['reviews']
    product_id = row['Product_ID']
    product = psycopg2.extensions.adapt(str(row['product']).encode('utf-8', 'replace')).getquoted().decode('utf-8')[1:-1]
    category = psycopg2.extensions.adapt(row['category'].encode('utf-8', 'replace')).getquoted().decode('utf-8')[1:-1]
    url = row['url']
      
    critical_review_id = row['Customer_id_Critical_Review'] if row['Customer_id_Critical_Review'] != 'None' else None
    customer_name = row['Customer_Name'] if row['Customer_Name'] != 'None' else None
    customer_review_comment = row['Customer_Review_Comment'] if row['Customer_Review_Comment'] != 'None' else None
    customer_review_title = row['Customer_Review_Title'] if row['Customer_Review_Title'] != 'None' else None
    customers_influenced = row['Customers_Influenced'] if row['Customers_Influenced'] != 'None' else None  # Correctly handle NaN values
    
    # Handle additional customer information
    customer_data = []
    for i in range(1, 6):
        customer_id = row[f'Customer_{i}_ID'] if row[f'Customer_{i}_ID'] != 'None' else None
        star_rating = row[f'Customer_{i}_Star_Rating'] if pd.notna(row[f'Customer_{i}_Star_Rating']) else None
        comment = psycopg2.extensions.adapt(str(row[f'Customer_{i}_Comment']).encode('utf-8', 'replace')).getquoted().decode('utf-8')[1:-1]
        buying_influence = row[f'Customer_{i}_buying_influence'] if pd.notna(row[f'Customer_{i}_buying_influence']) else None

        customer_data.extend([customer_id, star_rating, comment, buying_influence])

    # Validate and format the 'Post_Date' before returning

    # Validate and format the 'Post_Date' before returning
    post_date = row['Post_Date']
    if post_date is not None and isinstance(post_date, str):
        try:
            # Check if post_date is already in 'YYYY-MM-DD' format
            if re.match(r'\d{4}-\d{2}-\d{2}', post_date):
                datetime.strptime(post_date, '%Y-%m-%d')  # Validate the format
            else:
                # Try to parse the post_date to a datetime object with a different format
                parsed_date = datetime.strptime(post_date, '%Y-%m-%dT%H:%M:%S')
                # Format it back to a string in the desired format
                post_date = parsed_date.strftime('%Y-%m-%d')
        except ValueError as ve:
            logging.error(f"Invalid date format for Post_Date: {post_date}. Setting it to default value.")
            post_date = "0001-01-01"  # Default value for invalid date format
    else:
        logging.error(f"Post_Date is not a string or is None: {post_date}. Setting it to default value.")
        post_date = "0001-01-01"  # Default value for None or non-string values
    
    # Replace other None values with appropriate default values
    critical_review_id = critical_review_id if critical_review_id is not None else "Unavailable"
    customer_name = customer_name if customer_name is not None else "Unavailable"
    customer_review_comment = customer_review_comment if customer_review_comment is not None else "Unavailable"
    customer_review_title = customer_review_title if customer_review_title is not None else "Unavailable"
    customers_influenced = customers_influenced if customers_influenced is not None else 0  # Default numeric value
    
    # Handle additional customer information
    customer_data = []
    for i in range(1, 6):
        customer_id = row[f'Customer_{i}_ID'] if row[f'Customer_{i}_ID'] != 'None' else "Unavailable"
        star_rating = row[f'Customer_{i}_Star_Rating'] if pd.notna(row[f'Customer_{i}_Star_Rating']) else 0.0
        comment = psycopg2.extensions.adapt(str(row[f'Customer_{i}_Comment']).encode('utf-8', 'replace')).getquoted().decode('utf-8')[1:-1]
        comment = comment if comment != 'None' else "Unavailable"
        buying_influence = row[f'Customer_{i}_buying_influence'] if pd.notna(row[f'Customer_{i}_buying_influence']) else 0
        
        customer_data.extend([customer_id, star_rating, comment, buying_influence])

    logging.info(f"Formatted Post_Date: {post_date}")  # Log the formatted post_date
    
    return product_id, product, price, ratings, reviews, category, url, critical_review_id, customer_name, post_date, customer_review_comment, customer_review_title, customers_influenced, *customer_data


insert_query = """
INSERT INTO amazon_data_ext (
    product_id, product, price_dollars, star_ratings, total_ratings, category, url,
    Customer_id_Critical_Review, Customer_Name, Post_Date, Customer_Review_Comment, Customer_Review_Title, Customers_Influenced,
    """ + ", ".join([f"Customer_{i}_ID, Customer_{i}_Star_Rating, Customer_{i}_Comment, Customer_{i}_buying_influence" for i in range(1, 6)]) + """
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, """ + ", ".join(["%s"] * 20) + ")"


# Insert the data from the pandas DataFrame into the PostgreSQL table
for index, row in df.iterrows():
    try:
        cur.execute(insert_query, clean_format_data(row))
    except Exception as e:
        logging.error(f"Error inserting row at index {index}: {e}")
        logging.error(f"Row data: {row}")  # Log the entire row data
        conn.rollback()



conn.commit()
cur.close()
conn.close()

# Rename the columns in the DataFrame
df.rename(columns={'ratings': 'star_ratings', 'reviews': 'total_ratings', 'price': 'price_dollars'}, inplace=True)

# Save the DataFrame to a CSV file with updated column names
df.to_csv('amazon_data_ext.csv', index=False, encoding='utf-8')"



             
            # Remove the pattern "k out of 5 stars\n" from the comment
            actual_comment_title = re.sub(r'\d+(\.\d+)? out of 5 stars\n', '', actual_comment_title)

            result[f'Customer_{i+1}_Comment'] = actual_comment_title

            # Extract the Number of people who found the review helpful
            helpful_vote_tag = review_tag.select_one('span[data-hook="helpful-vote-statement"]')
            helpful_count = w2n.word_to_num(helpful_vote_tag.text.split()[0]) if helpful_vote_tag else 0
            result[f'Customer_{i+1}_buying_influence'] = helpful_count
        
            # Extract all elements matching the criteria
            critical_review_tags = soup.select('div[id^="viewpoint-"]')
            # critical_review_tags = soup.select('div.a-column.a-span6.view-point-review.critical-review.a-span-last')
            if len(critical_review_tags) > 1:
                # If there is more than one matching element, select the second one
                critical_review_tag = critical_review_tags[1]
                critical_review_tag_pack = critical_review_tag.get('id', 'NaN').replace('viewpoint-', '')
                result['Customer_id_Critical_Review'] = critical_review_tag_pack
                
                # Extract Customer_Name
                customer_name_tags = soup.select('span.a-profile-name')
                result['Customer_Name'] = customer_name_tags[1].text if len(customer_name_tags) > 1 else 'NaN'
                    
                # Extract Customer_Review_Comment                
                review_comment_tag = critical_review_tag.find('div', class_='a-row a-spacing-top-mini')
                result['Customer_Review_Comment'] = review_comment_tag.text.strip() if review_comment_tag else 'NaN'
                
                # Extract Customer_Review_Title
                review_title_tag = critical_review_tag.select_one('span[data-hook="review-title"]')
                result['Customer_Review_Title'] = review_title_tag.text if review_title_tag else 'NaN'

                # Extract the post time
                critical_review_tags_date = critical_review_tag.select('div.a-expander-content.a-expander-partial-collapse-content span.a-size-base.a-color-secondary.review-date')
                if critical_review_tags_date:
                    post_time_text = critical_review_tags_date[0].text.strip()
                    match = re.search(r'on (.+)$', post_time_text)
                    if match:
                        date_string = match.group(1)
                        try:
                            post_date = datetime.strptime(date_string, '%B %d, %Y')
                            # Convert the datetime object to a string in ISO format
                            result['Post_Date'] = post_date.isoformat()                            
                        except ValueError as ve:
                            print(f"Error parsing date string {date_string}: {ve}")
                            result['Post_Date'] = None
                    else:
                        print("Date not found in text:", post_time_text)
                        result['Post_Date'] = None
                else:
                    print("Date tag not found")
                    result['Post_Date'] = None

                # Use soup.select() to find all matching elements
                critical_review_tags_ = soup.select('div.a-column.a-span6.view-point-review.critical-review.a-span-last div.a-row.a-spacing-top-small span.a-size-small.a-color-tertiary span.review-votes')


                # Check if any elements were found
                if critical_review_tags_:
                    # Take the first found element (if there are multiple) and directly extract the text
                    helpful_text = critical_review_tags_[0].text.strip()
                    print("Helpful Text:", helpful_text)  # Debugging line
                    
                    # Check if the text starts with a digit and extract the first contiguous digit sequence
                    match = re.match(r'\d+', helpful_text)
                    if match:
                        helpful_count = int(match.group())
                    else:
                        # If the text doesn't start with a digit, try converting the first word to a number
                        helpful_count = w2n.word_to_num(helpful_text.split()[0])
                else:
                    print("Tag not found")  # Debugging line
                    helpful_count = 0

                result['Customers_Influenced'] = helpful_count


            else:
                # Handle the case where there is only one or no matching element
                result['Customer_id_Critical_Review'] = 'NaN'
                result['Customer_Name'] = 'NaN'
                result['Customer_Review_Comment'] = 'NaN'
                result['Customer_Review_Title'] = 'NaN'
                result['Customers_Influenced'] = 0

        return result
    except Exception as e:
        print(f"Error scraping extra parameters for {url}: {e}")
        traceback.print_exc()
    return {}

